In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [3]:
df = pd.read_csv('/Users/tikaramadhini/Documents/theraplydata/thedata/final list of therapists and other info.csv', encoding_errors='ignore', sep=';', index_col='Number')

In [4]:
df.head(5)

,Name,Gender,PersonalTherapy,Group Therapy,Language,TypeOfTherapy,Expertise,Insurance,PatientType,Address,PostalCode,PhoneNumber,Email,Website,CallAvailability,BarrierFreeAccess,WaitingTime(Weeks)
Number,,,,,,,,,,,,,,,,,
1,Prof.Dr.Dr. Rainer Kaus,Male,Yes,Yes,"German, English, Hebrew, Arabic","analytical psychotherapy, depth psychology-bas...","Depression, Panic Attack, Burnout, Crisis Inte...","Public, Private","Adult, Kids and Teenagers",Garnisonkirchplatz 2/II,10178,030 24727950,NaN,NaN,Monday 15.00-16.40,Yes,4
2,Dr. Hannsi Padberg,Female,Yes,Yes,German,depth psychology-based psychotherapie,"Depression, Panic Attack, Burnout, Crisis Inte...","Public, Private","Adult, Kids and Teenagers",Garnisonkirchplatz 2/II,10178,030 24727950,NaN,NaN,Monday 15.00-16.40,Yes,2
3,Dipl.-Psych. Reiner Domes-Nontasila,Male,Yes,No,"German, English",depth psychology-based psychotherapie,"fear - phobia, depression, eating disorder, em...","Public, Private",Adult,Dircksenstrae 47,10178,030 55494880,NaN,NaN,"Monday 14:10 - 15:00, Tuesday 08:10 - 09:00, W...",No,5
4,Dr. Walter Raffauf,Male,Yes,No,"German, English, French, Italian, Dutch, Russian",depth psychology-based psychotherapie,NaN,"Public, Private",Adult,Dircksenstrae 47,10178,030 234569870,NaN,http://www.neuropraxis-mitte.de,,No,6
5,Dipl.-Psych. Frank Taffelt,Male,Yes,No,German,"Analytical psychotherapy, depth psychology-bas...","fear - phobia, depression, eating disorder, em...","Public, Private","Adult, Kids and Teenagers",Rosenthaler Strae 40/41,10178,030 2829012,h.f.taffelt@t-online.de,NaN,"Monday 10:20 - 10:45, 14:50 - 15:15 Tuesday 10...",Yes,8


In [19]:
df['similartherapist'] = df['Expertise'] + df['TypeOfTherapy']
df['similartherapist'] = df['similartherapist'].fillna('')

In [66]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['similartherapist'])

In [7]:
tfidf_matrix.shape

(130, 355)

In [8]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
cosine_sim[0]

array([1.        , 0.61021375, 0.14256276, 0.        , 0.26464517,
       0.12100111, 0.14256276, 0.27910663, 0.3097165 , 0.03952085,
       0.26464517, 0.        , 0.29321396, 0.3097165 , 0.29321396,
       0.29321396, 0.09132385, 0.09660676, 0.0190235 , 0.0286848 ,
       0.03321448, 0.27910663, 0.27910663, 0.03321448, 0.14256276,
       0.61021375, 0.27910663, 0.27910663, 0.26464517, 0.26464517,
       0.26464517, 0.03321448, 0.14636174, 0.14256276, 0.03321448,
       0.27910663, 0.14636174, 0.21287485, 0.14256276, 0.        ,
       0.14636174, 0.14256276, 0.05118231, 0.03677547, 0.19257737,
       0.14636174, 0.03321448, 0.53887183, 0.03321448, 0.27910663,
       0.0190158 , 0.03162485, 0.03113196, 0.02257494, 0.0790871 ,
       0.10549018, 0.08735544, 0.02149347, 0.02662664, 0.02330707,
       0.04105039, 0.00467701, 0.03269143, 0.03939942, 0.1272724 ,
       0.01293631, 0.12583912, 0.01702283, 0.03762086, 0.1272724 ,
       0.01873578, 0.01775711, 0.01751587, 0.01588237, 0.04446

In [10]:
df = df.reset_index()
names = df['Name']
indexes = pd.Series(df.index, index=df['Name'])

In [82]:
def get_recommendations(Name):
    idx = indexes[Name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    therapist_indexes = [i[0] for i in sim_scores]
    return names.iloc[therapist_indexes]

In [84]:
get_recommendations('Dipl.-Med. Katharina Bolze')

36                  Dr. Daniel Schumacher
40              Dipl.-Psych. Heika Grning
45             Dipl.-Med. Katharina Bolze
66    Dipl.-Psych.Alexandra von der Linde
64              M.Sc.Psych.Jonas Schndorf
69                           Lutz Ulbrich
7                         Dr. Andrea Dorn
21                 Dipl.-Psych. Anja Voss
22           Dipl.-Psych. Monika Englisch
26                     Dr. Wilhelm Brggen
Name: Name, dtype: object